<a href="https://colab.research.google.com/github/marcory-hub/yolo11n-on-grove-vision-ai-v2/blob/main/YOLO11n_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO11n training




1. Make sure images and labels from your dataset have this folder structure with these exact names. And add `data.yaml` to main folder.

```
🗂️ dataset
  🗂️ train
    🗂️ images
    🗂️ labels
  🗂️ valid
    🗂️ images
    🗂️ labels
  data.yaml
```

2. Zip the dataset folder to a file names `dataset.zip` On mac use `zip -r dataset.zip . -x "*.DS_Store" "__MACOSX/*" ".Trashes/*" ".Spotlight-V100/*" ".TemporaryItems/*"` to exclude hidden files, such as finderfiles, from the zipped file.

3. Make in /content/drive/MyDrive the folder `yolo`. Copy the `dataset.zip` file to this folder, it is needed to make a callibration image set and your yolo model, fe `best.pt` to this folder. For the model you can use a custom name and adjust it in the options below.

4. Copy the `dataset.zip` file to the folder /content/drive/MyDrive/yolo.


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# Adjust the zipped file name

In [ ]:
# Copy zipped dataset to colab and unzip the dataset
!scp '/content/drive/MyDrive/yolo/dataset.zip' '/content/dataset.zip'
!unzip '/content/dataset.zip' -d '/content/'

In [ ]:
# Install the required packages for Ultralytics YOLO 8.3.166 and Weights & Biases
!pip install -U ultralytics wandb

# Enable W&B logging for Ultralytics
!yolo settings wandb=True

In [ ]:
# check if ultralytics version is 8.3.166
import ultralytics
print(ultralytics.__version__)

In [ ]:
import wandb
from google.colab import userdata

# Retrieve the WandB API key from Colab's user secrets
api_key = userdata.get('wandb-key')

# Log in to WandB
wandb.login(key=api_key)

In [ ]:
from ultralytics import YOLO

# Initialize YOLO
yolo = YOLO()

# Enable W&B logging for Ultralytics
yolo.settings = {"wandb": True}


# Train, zip and download the model

Common batch sizes that work well with GPU architectures are powers of two, such as 16, 32, 64, 128, ... 512. -1 for autobatch.


In [ ]:
from ultralytics import YOLO
from google.colab import files

name = "best" #@param {type:"string"}
project = "project" #@param {type:"string"}
epochs = 10 #@param {type:"integer"}
batch_size = 128 #@param {type:"integer"}
imgsz = 224 #@param {type:"integer"}
dataset_path = "/content/dataset/data.yaml" #@param {type:"string"}

# Get pre-trained model
model = YOLO("yolo11n.pt")

# Train/fine-tune model
model.train(data=dataset_path,
            augment=True,
            epochs=epochs,
            patience=10,
            imgsz=imgsz,
            batch=batch_size,
            project=project,
            name=name)

# zip and download
!zip -r /content/{name}.zip /content/{project}
files.download(f'/content/{name}.zip')

-----